# Import modules

In [18]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

In [20]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_data = """query data ($nationality: String!, $job: String!)
            {movieList(nationality: $nationality, job: $job)
                {
                title 
                revenue
                }}"""
query_introspect = """{
  __schema {
    types {
      name
    }
  }
}"""
query_playground = """{"operationName":"IntrospectionQuery","variables":{},"query":"query IntrospectionQuery {\n  __schema {\n    queryType {\n      name\n    }\n    mutationType {\n      name\n    }\n    subscriptionType {\n      name\n    }\n    types {\n      ...FullType\n    }\n    directives {\n      name\n      description\n      locations\n      args {\n        ...InputValue\n      }\n    }\n  }\n}\n\nfragment FullType on __Type {\n  kind\n  name\n  description\n  fields(includeDeprecated: true) {\n    name\n    description\n    args {\n      ...InputValue\n    }\n    type {\n      ...TypeRef\n    }\n    isDeprecated\n    deprecationReason\n  }\n  inputFields {\n    ...InputValue\n  }\n  interfaces {\n    ...TypeRef\n  }\n  enumValues(includeDeprecated: true) {\n    name\n    description\n    isDeprecated\n    deprecationReason\n  }\n  possibleTypes {\n    ...TypeRef\n  }\n}\n\nfragment InputValue on __InputValue {\n  name\n  description\n  type {\n    ...TypeRef\n  }\n  defaultValue\n}\n\nfragment TypeRef on __Type {\n  kind\n  name\n  ofType {\n    kind\n    name\n    ofType {\n      kind\n      name\n      ofType {\n        kind\n        name\n        ofType {\n          kind\n          name\n          ofType {\n            kind\n            name\n            ofType {\n              kind\n              name\n              ofType {\n                kind\n                name\n              }\n            }\n          }\n        }\n      }\n    }\n  }\n}\n"}""".replace('\n', '')

In [23]:
input_query_data = json.dumps({'variables': {'nationality': 'English', 'job': 'Director'}, 'query':query_data})
input_query_metadata = json.dumps({'query':query_metadata})

In [24]:
input_query_metadata

'{"query": "query metadata {nationalityList, jobList}"}'

# Test handler functions

In [25]:
import handler 
importlib.reload(handler);

In [29]:
%%time
handler.handle(query_playground)

CPU times: user 35.1 ms, sys: 391 µs, total: 35.5 ms
Wall time: 35.2 ms


'{"__schema": {"queryType": {"name": "Query"}, "mutationType": null, "subscriptionType": null, "types": [{"kind": "OBJECT", "name": "Query", "description": null, "fields": [{"name": "nationalityList", "description": null, "args": [], "type": {"kind": "NON_NULL", "name": null, "ofType": {"kind": "LIST", "name": null, "ofType": {"kind": "SCALAR", "name": "String", "ofType": null}}}, "isDeprecated": false, "deprecationReason": null}, {"name": "jobList", "description": null, "args": [], "type": {"kind": "NON_NULL", "name": null, "ofType": {"kind": "LIST", "name": null, "ofType": {"kind": "SCALAR", "name": "String", "ofType": null}}}, "isDeprecated": false, "deprecationReason": null}, {"name": "movieList", "description": null, "args": [{"name": "nationality", "description": null, "type": {"kind": "SCALAR", "name": "String", "ofType": null}, "defaultValue": "\\"English\\""}, {"name": "job", "description": null, "type": {"kind": "SCALAR", "name": "String", "ofType": null}, "defaultValue": "\\

In [283]:
with open(str(bundle_root / 'functions/index_by_nationality/schema.json'), 'w') as fp:
    json.dump(handler.schema.introspect(), fp)

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [263]:
function_url = 'https://labs.cloud.contiamo.com/21b899ae-3eba-56a7-97da-fa26c050d1c5/v1/function/26a6b514-0324-4078-abc7-953d851c3aa4/index_by_nationality'

In [289]:
%%time
deployed_output = requests.post(function_url,
                                data=query_introspect).json()

CPU times: user 20.2 ms, sys: 0 ns, total: 20.2 ms
Wall time: 215 ms


In [290]:
deployed_output

{'__schema': {'types': [{'name': 'Query'},
   {'name': 'String'},
   {'name': 'movie'},
   {'name': 'Int'},
   {'name': '__Schema'},
   {'name': '__Type'},
   {'name': '__TypeKind'},
   {'name': 'Boolean'},
   {'name': '__Field'},
   {'name': '__InputValue'},
   {'name': '__EnumValue'},
   {'name': '__Directive'},
   {'name': '__DirectiveLocation'}]}}

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True